In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd

In [ ]:
training_df = pd.read_csv("Datasets/TrainingWiDS2021.csv")
training_df.sample(3)

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
99469,99470,150589,96,82.0,20.989855,0,Caucasian,M,167.6,Emergency Department,Accident & Emergency,978,admit,Med-Surg ICU,0.166667,0,58.96,NaN,306.0,802.01,0,0,NaN,38.0,2.10,NaN,4.0,6.0,0.0,4.0,108.0,124.0,18.1,0,52.0,NaN,NaN,NaN,NaN,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.1,4.1,132.0,132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
92457,92458,220242,184,84.0,40.143141,0,Caucasian,M,168.0,NaN,Accident & Emergency,907,admit,Med-Surg ICU,0.048611,0,113.30,2.9,113.0,501.05,0,0,0.2,12.0,1.37,NaN,4.0,6.0,0.0,5.0,NaN,56.0,29.5,0,61.0,NaN,NaN,NaN,NaN,33.0,...,NaN,NaN,NaN,NaN,2.1,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
70601,70602,229950,185,NaN,24.230661,0,Caucasian,M,177.8,NaN,Accident & Emergency,711,admit,CCU-CTICU,0.075694,0,76.60,3.6,302.0,109.02,0,0,0.6,35.0,1.60,NaN,4.0,6.0,0.0,5.0,111.0,107.0,36.8,0,59.0,NaN,NaN,NaN,NaN,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [ ]:
training_df.shape

(130157, 181)

In [ ]:
pd.set_option('display.max_rows', None)
training_df.isnull().sum() / training_df.shape[0] * 100

Unnamed: 0                      0.000000
encounter_id                    0.000000
hospital_id                     0.000000
age                             3.832295
bmi                             3.449680
elective_surgery                0.000000
ethnicity                       1.219297
gender                          0.050708
height                          1.595765
hospital_admit_source          25.506120
icu_admit_source                0.184393
icu_id                          0.000000
icu_stay_type                   0.000000
icu_type                        0.000000
pre_icu_los_days                0.000000
readmission_status              0.000000
weight                          2.660633
albumin_apache                 60.052859
apache_2_diagnosis              1.294590
apache_3j_diagnosis             0.664582
apache_post_operative           0.000000
arf_apache                      0.000000
bilirubin_apache               63.431087
bun_apache                     19.523345
creatinine_apach

In [ ]:
percent_missing_values_attributes = training_df.isnull().sum() / training_df.shape[0] * 100
missing_values_attributes = []
for item,value in percent_missing_values_attributes.items():
    if value >= 50:
        missing_values_attributes.append(item)
len(missing_values_attributes)

73

In [ ]:
position_of_glu = []
for i in range(len(missing_values_attributes)):
    if missing_values_attributes[i].find("glu") > -1:
        position_of_glu.append(i)
for i in range(len(position_of_glu)):
    missing_values_attributes.pop(position_of_glu[i]-i)
missing_values_attributes

['albumin_apache',
 'bilirubin_apache',
 'fio2_apache',
 'paco2_apache',
 'paco2_for_ph_apache',
 'pao2_apache',
 'ph_apache',
 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min',
 'd1_mbp_invasive_max',
 'd1_mbp_invasive_min',
 'd1_sysbp_invasive_max',
 'd1_sysbp_invasive_min',
 'h1_diasbp_invasive_max',
 'h1_diasbp_invasive_min',
 'h1_mbp_invasive_max',
 'h1_mbp_invasive_min',
 'h1_sysbp_invasive_max',
 'h1_sysbp_invasive_min',
 'd1_albumin_max',
 'd1_albumin_min',
 'd1_bilirubin_max',
 'd1_bilirubin_min',
 'd1_inr_max',
 'd1_inr_min',
 'd1_lactate_max',
 'd1_lactate_min',
 'h1_albumin_max',
 'h1_albumin_min',
 'h1_bilirubin_max',
 'h1_bilirubin_min',
 'h1_bun_max',
 'h1_bun_min',
 'h1_calcium_max',
 'h1_calcium_min',
 'h1_creatinine_max',
 'h1_creatinine_min',
 'h1_hco3_max',
 'h1_hco3_min',
 'h1_hemaglobin_max',
 'h1_hemaglobin_min',
 'h1_hematocrit_max',
 'h1_hematocrit_min',
 'h1_inr_max',
 'h1_inr_min',
 'h1_lactate_max',
 'h1_lactate_min',
 'h1_platelets_max',
 'h1_platelets_m

In [ ]:
training_df = training_df.drop(columns = missing_values_attributes)

In [ ]:
training_df = training_df.drop(columns = ['Unnamed: 0', 'hospital_id'])

In [ ]:
training_df.isnull().sum() / training_df.shape[0] * 100

encounter_id                    0.000000
age                             3.832295
bmi                             3.449680
elective_surgery                0.000000
ethnicity                       1.219297
gender                          0.050708
height                          1.595765
hospital_admit_source          25.506120
icu_admit_source                0.184393
icu_id                          0.000000
icu_stay_type                   0.000000
icu_type                        0.000000
pre_icu_los_days                0.000000
readmission_status              0.000000
weight                          2.660633
apache_2_diagnosis              1.294590
apache_3j_diagnosis             0.664582
apache_post_operative           0.000000
arf_apache                      0.000000
bun_apache                     19.523345
creatinine_apache              19.116913
gcs_eyes_apache                 1.682583
gcs_motor_apache                1.682583
gcs_unable_apache               0.544727
gcs_verbal_apach

In [ ]:
training_df.head(5)

,encounter_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,214826,68.0,22.732803,0,Caucasian,M,180.3,Floor,Floor,92,admit,CTICU,0.541667,0,73.9,2.3,113.0,502.01,0,0,0.4,31.0,2.51,NaN,3.0,6.0,0.0,4.0,168.0,118.0,27.4,0,40.0,NaN,NaN,NaN,NaN,36.0,134.0,39.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
1,246060,77.0,27.421875,0,Caucasian,F,160.0,Floor,Floor,90,admit,Med-Surg ICU,0.927778,0,70.2,NaN,108.0,203.01,0,0,NaN,9.0,0.56,1.0,1.0,3.0,0.0,1.0,145.0,120.0,36.9,0,46.0,37.0,37.0,51.0,7.45,33.0,145.0,35.1,...,11.3,11.3,36.9,36.9,1.3,1.3,3.5,3.5,557.0,557.0,4.2,4.2,145.0,145.0,12.7,12.7,37.0,37.0,7.45,7.45,51.0,51.0,54.8,51.000000,37.0,37.0,7.45,7.45,51.0,51.0,51.0,51.0,0,0,0,0,0,0,0,1
2,276985,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,93,admit,Med-Surg ICU,0.000694,0,95.3,NaN,122.0,703.03,0,0,NaN,NaN,NaN,NaN,3.0,6.0,0.0,5.0,NaN,102.0,NaN,0,68.0,NaN,NaN,NaN,NaN,37.0,NaN,36.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
3,262220,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,92,admit,CTICU,0.000694,0,61.7,NaN,203.0,1206.03,1,0,NaN,NaN,NaN,0.6,4.0,6.0,0.0,5.0,185.0,114.0,25.9,1,60.0,30.0,30.0,142.0,7.39,4.0,NaN,34.8,...,11.6,11.6,34.0,34.0,1.6,1.1,NaN,NaN,43.0,43.0,NaN,NaN,NaN,NaN,8.8,8.8,37.0,27.0,7.44,7.34,337.0,102.0,342.5,236.666667,36.0,33.0,7.37,7.34,337.0,265.0,337.0,337.0,0,0,0,0,0,0,0,0
4,201746,19.0,NaN,0,Caucasian,M,188.0,NaN,Accident & Emergency,91,admit,Med-Surg ICU,0.073611,0,NaN,NaN,119.0,601.01,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,0,103.0,NaN,NaN,NaN,NaN,16.0,NaN,36.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


# **Filling continuous variables**

In [ ]:
from sklearn.impute import KNNImputer
import numpy as np
def fill_missing_value_for_continuous_atrr(df, name_attribute):
    imputer = KNNImputer(n_neighbors = 5, weights='distance')
    return imputer.fit_transform(df[name_attribute].to_numpy().reshape(-1,1))

In [ ]:
for column in training_df.columns:
    if training_df[column].isnull().sum() > 0 and str(training_df[column].dtype).find('64') > -1:
        training_df[column] = fill_missing_value_for_continuous_atrr(training_df, column)

In [ ]:
training_df.to_csv("training_continuous_filling.csv", index = False)

In [ ]:
training_df = pd.read_csv("training_continuous_filling.csv")

In [ ]:
training_df.shape

# Filling category variables

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder
from fancyimpute import KNN
encoder = OrdinalEncoder()
imputer = IterativeImputer(ExtraTreesRegressor())
def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data
def fill_missing_value_for_categorical_atrr(df):
    return pd.DataFrame(np.round(imputer.fit_transform(df.values.reshape(-1,1))))

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [ ]:
training_df.sample(3)

,encounter_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
16222,167535,63.0,20.976562,0,Hispanic,M,160.0,NaN,Accident & Emergency,267,admit,Med-Surg ICU,0.054861,0,53.7,3.3,124.0,305.02,0,0,1.2,127.0,11.18,NaN,4.0,6.0,0.0,5.0,148.0,138.0,27.9,0,44.0,NaN,NaN,NaN,NaN,35.0,129.0,36.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.9,5.9,132.0,132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
95129,205997,81.0,28.338765,0,Caucasian,M,182.9,NaN,Accident & Emergency,976,admit,Med-Surg ICU,0.247222,0,94.8,NaN,113.0,502.01,0,0,NaN,27.0,1.97,NaN,4.0,6.0,0.0,4.0,91.0,107.0,31.6,0,136.0,NaN,NaN,NaN,NaN,28.0,140.0,36.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
116524,271324,88.0,23.475546,0,Caucasian,F,155.0,Emergency Department,Accident & Emergency,1032,admit,Med-Surg ICU,0.308333,0,56.4,2.8,113.0,501.02,0,0,0.5,14.0,0.61,NaN,4.0,6.0,0.0,4.0,80.0,99.0,37.1,0,45.0,NaN,NaN,NaN,NaN,27.0,137.0,37.1,...,NaN,NaN,NaN,NaN,1.3,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0


In [ ]:
training_df['ethnicity'] = encode(training_df['ethnicity'])
training_df['gender'] = encode(training_df['gender'])
training_df['hospital_admit_source'] = encode(training_df['hospital_admit_source'])
training_df['icu_admit_source'] = encode(training_df['icu_admit_source'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [ ]:
training_df['ethnicity'] = fill_missing_value_for_categorical_atrr(training_df['ethnicity'])
training_df['gender'] = fill_missing_value_for_categorical_atrr(training_df['gender'])
training_df['hospital_admit_source'] = fill_missing_value_for_categorical_atrr(training_df['hospital_admit_source'])
training_df['icu_admit_source'] = fill_missing_value_for_categorical_atrr(training_df['icu_admit_source'])

In [ ]:
pd.set_option('display.max_columns', None)
training_df.sample(10)

In [ ]:
from sklearn import preprocessing
icu_stay_type_le = preprocessing.LabelEncoder().fit(training_df['icu_stay_type'])
icu_type_le = preprocessing.LabelEncoder().fit(training_df['icu_type'])
# training_df['icu_stay_type'] = preprocessing.LabelEncoder().fit_transform(training_df['icu_stay_type'])
# training_df['icu_type'] = preprocessing.LabelEncoder().fit_transform(training_df['icu_type'])
training_df['icu_stay_type'] = icu_stay_type_le.transform(training_df['icu_stay_type'])
training_df['icu_type'] = icu_type_le.transform(training_df['icu_type'])

In [ ]:
training_df.isnull().sum() / training_df.shape[0] * 100

encounter_id                   0.0
age                            0.0
bmi                            0.0
elective_surgery               0.0
ethnicity                      0.0
gender                         0.0
height                         0.0
hospital_admit_source          0.0
icu_admit_source               0.0
icu_id                         0.0
icu_stay_type                  0.0
icu_type                       0.0
pre_icu_los_days               0.0
readmission_status             0.0
weight                         0.0
apache_2_diagnosis             0.0
apache_3j_diagnosis            0.0
apache_post_operative          0.0
arf_apache                     0.0
bun_apache                     0.0
creatinine_apache              0.0
gcs_eyes_apache                0.0
gcs_motor_apache               0.0
gcs_unable_apache              0.0
gcs_verbal_apache              0.0
glucose_apache                 0.0
heart_rate_apache              0.0
hematocrit_apache              0.0
intubated_apache    

In [ ]:
training_df.sample(10)

,encounter_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
85671,197825,67.0,33.159778,1,2,1,172.7,8,2,841,0,7,0.274306,0,98.90,NaN,203.0,1206.03,1,0,NaN,18.0,0.80,NaN,4.0,6.0,0.0,5.0,210.0,99.0,32.2,0,122.0,NaN,NaN,NaN,NaN,35.0,139.0,36.5,...,13.1,13.1,37.0,37.0,1.5,1.5,NaN,NaN,85.0,85.0,3.7,3.5,142.0,140.0,9.6,9.6,47.0,41.0,NaN,NaN,63.0,50.0,157.500000,83.333333,47.0,47.0,NaN,NaN,50.0,50.0,83.333333,83.333333,0,0,0,0,0,0,0,1
97020,264506,61.0,44.492188,0,0,0,160.0,NaN,0,998,0,5,0.218056,0,113.90,NaN,305.0,901.03,0,0,NaN,53.0,7.16,0.40,4.0,6.0,0.0,4.0,219.0,158.0,27.1,1,155.0,30.2,30.2,189.0,7.218,29.0,137.0,37.0,...,NaN,NaN,NaN,NaN,1.1,1.1,1.7,1.6,NaN,NaN,3.5,3.2,138.0,137.0,NaN,NaN,30.2,30.2,7.218,7.218,189.0,189.0,472.500000,472.500000,30.2,30.2,7.218,7.218,189.0,189.0,472.500000,472.500000,0,0,0,0,0,0,0,0
7604,161529,44.0,21.625386,0,3,1,180.3,3,0,114,0,0,0.004861,0,70.30,2.9,123.0,702.01,0,0,0.9,51.0,3.81,NaN,4.0,6.0,0.0,5.0,598.7,126.0,36.5,0,135.0,NaN,NaN,NaN,NaN,31.0,131.0,39.0,...,NaN,NaN,NaN,NaN,1.1,1.1,NaN,NaN,NaN,NaN,4.0,4.0,131.0,131.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1
34855,151216,74.0,17.108487,0,2,0,167.5,3,0,445,0,5,0.000694,0,48.00,NaN,106.0,212.01,0,0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,5.0,122.0,102.0,NaN,0,47.0,NaN,NaN,NaN,NaN,50.0,NaN,36.5,...,11.2,11.2,37.2,37.2,NaN,NaN,1.8,1.7,172.0,172.0,4.4,4.4,134.0,134.0,9.4,9.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
80230,211207,59.0,28.291796,0,2,0,162.6,NaN,0,775,2,5,0.000694,0,74.80,3.4,113.0,502.01,0,0,1.1,16.0,1.08,NaN,1.0,1.0,0.0,1.0,124.0,102.0,32.5,0,55.0,NaN,NaN,NaN,NaN,27.0,138.0,35.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0
103139,264242,43.0,27.985600,0,2,1,175.3,3,0,998,0,5,0.247917,0,86.00,2.1,123.0,702.01,0,0,0.2,40.0,3.59,0.28,3.0,6.0,0.0,4.0,598.7,107.0,20.0,1,62.0,33.6,33.6,54.0,7.359,4.0,117.0,35.1,...,NaN,NaN,NaN,NaN,NaN,NaN,9.6,9.6,NaN,NaN,5.6,5.2,114.0,114.0,NaN,NaN,33.6,23.8,7.359,6.890,127.0,54.0,453.571429,192.857143,23.8,23.8,6.930,6.900,82.0,82.0,205.000000,205.000000,0,0,0,0,0,0,0,1
57115,179559,65.0,20.198122,0,2,1,170.2,3,0,620,0,5,0.215972,0,58.51,NaN,120.0,407.01,0,0,NaN,13.0,1.08,0.80,4.0,6.0,0.0,1.0,137.0,61.0,44.3,1,64.0,41.0,41.0,200.0,7.420,12.0,142.0,36.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.0,39.0,7.420,7.420,200.0,79.0,250.000000,225.714286,41.0,41.0,7.420,7.420,200.0,200.0,250.000000,250.000000,0,0,0,0,0,0,0,0
128237,192415,35.0,42.811791,0,2,0,157.5,NaN,1,1104,0,3,1.044444,0,106.20,3.7,301.0,410.01,0,0,0.4,9.0,0.72,0.25,3.0,6.0,0.0,4.0,83.0,98.0,33.0,0,64.0,55.0,55.0,71.0,7.382,47.0,141.0,36.7,...,10.9,10.9,32.0,32.0,NaN,NaN,NaN,NaN,

# Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection       import train_test_split
from sklearn.preprocessing         import StandardScaler
from sklearn.preprocessing         import RobustScaler
from sklearn.preprocessing         import QuantileTransformer
from sklearn.preprocessing         import PowerTransformer
from sklearn.preprocessing         import MinMaxScaler
from sklearn.preprocessing         import MaxAbsScaler
from sklearn.pipeline              import Pipeline
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.svm                   import SVC
from sklearn.tree                  import DecisionTreeClassifier
from sklearn.ensemble              import RandomForestClassifier
from sklearn.naive_bayes           import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model          import LogisticRegression
from sklearn.metrics               import mean_squared_error
from sklearn.model_selection       import cross_val_score
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

from sklearn.metrics import accuracy_score

In [ ]:
scalers = [['standard',StandardScaler()],
           ['robust',RobustScaler()],
           ['quantile',QuantileTransformer()],
           ['power',PowerTransformer()],
           ['minmax',MinMaxScaler()],
           ['maxabs',MaxAbsScaler()]]
models =  [['Xgboost',DecisionTreeClassifier()],
           ['RF',RandomForestClassifier()],
           ['GNB',GaussianNB()],
           ['LDA',LinearDiscriminantAnalysis()],
           ['LR',LogisticRegression()]]

In [ ]:
training_df['ethnicity'] = training_df['ethnicity'].astype('float64')
training_df['gender'] = training_df['gender'].astype("float64")
training_df['hospital_admit_source'] = training_df['hospital_admit_source'].astype("float64")
training_df['icu_admit_source'] = training_df['icu_admit_source'].astype('float64')

In [ ]:
X_train = training_df.iloc[:,:-1]
y_train = training_df.iloc[:,-1]

In [ ]:
X = training_df.iloc[:,:-1]
y = training_df.iloc[:,-1]

In [ ]:
robust = RobustScaler().fit(X_train)
X_train = robust.transform(X_train)

In [ ]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="aucpr")

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X, y)

report_best_scores(search.cv_results_, 1)

In [ ]:
report_best_scores(search.cv_results_, 1)

In [ ]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="aucpr")


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:

xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)])

y_pred = xgb_model.predict(X_test)

accuracy_score(y_test, y_pred)

[0]	validation_0-aucpr:0.492336
Will train until validation_0-aucpr hasn't improved in 5 rounds.
[1]	validation_0-aucpr:0.508393
[2]	validation_0-aucpr:0.510218
[3]	validation_0-aucpr:0.516544
[4]	validation_0-aucpr:0.519091
[5]	validation_0-aucpr:0.52046
[6]	validation_0-aucpr:0.523865
[7]	validation_0-aucpr:0.526667
[8]	validation_0-aucpr:0.527998
[9]	validation_0-aucpr:0.528553
[10]	validation_0-aucpr:0.529879
[11]	validation_0-aucpr:0.534942
[12]	validation_0-aucpr:0.535617
[13]	validation_0-aucpr:0.536158
[14]	validation_0-aucpr:0.539962
[15]	validation_0-aucpr:0.545198
[16]	validation_0-aucpr:0.547363
[17]	validation_0-aucpr:0.547855
[18]	validation_0-aucpr:0.549495
[19]	validation_0-aucpr:0.55231
[20]	validation_0-aucpr:0.553544
[21]	validation_0-aucpr:0.554731
[22]	validation_0-aucpr:0.555028
[23]	validation_0-aucpr:0.556085
[24]	validation_0-aucpr:0.558345
[25]	validation_0-aucpr:0.558992
[26]	validation_0-aucpr:0.559403
[27]	validation_0-aucpr:0.5609
[28]	validation_0-aucpr:0

0.8300553165334973

In [ ]:
print("Shape of X training set: ",X_train.shape)

Shape of X training set:  (130157, 107)


In [ ]:
for model in models:
  for scaler in scalers:
    name = model[0]+'_'+scaler[0]
    print(name)
    pipe = Pipeline([('scaler', scaler[1]), ('model', model[1])], verbose=True)
    pipe.fit(X_train, y_train)
    joblib.dump(pipe,'Models/'+name+'.pkl')

DT_standard
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.3s
[Pipeline] ............. (step 2 of 2) Processing model, total=  15.1s
DT_robust
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.6s
[Pipeline] ............. (step 2 of 2) Processing model, total=  15.1s
DT_quantile
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   3.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=  15.1s
DT_power


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:216: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


[Pipeline] ............ (step 1 of 2) Processing scaler, total=  28.6s
[Pipeline] ............. (step 2 of 2) Processing model, total=  15.1s
DT_minmax
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=  15.2s
DT_maxabs
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=  15.3s
RF_standard
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.3s
[Pipeline] ............. (step 2 of 2) Processing model, total= 1.6min
RF_robust
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.6s
[Pipeline] ............. (step 2 of 2) Processing model, total= 1.6min
RF_quantile
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   3.1s
[Pipeline] ............. (step 2 of 2) Processing model, total= 1.6min
RF_power


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:216: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


[Pipeline] ............ (step 1 of 2) Processing scaler, total=  29.0s
[Pipeline] ............. (step 2 of 2) Processing model, total= 1.6min
RF_minmax
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total= 1.6min
RF_maxabs
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total= 1.6min
GNB_standard
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.3s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
GNB_robust
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.6s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
GNB_quantile
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   3.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   0.2s
GNB_power


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[Pipeline] ............. (step 2 of 2) Processing model, total=   4.3s
LR_minmax
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.2s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[Pipeline] ............. (step 2 of 2) Processing model, total=   4.7s
LR_maxabs
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.1s
[Pipeline] ............. (step 2 of 2) Processing model, total=   4.6s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Test preprocessing data

In [ ]:
test_df = pd.read_csv("Datasets/UnlabeledWiDS2021.csv")
test_df.sample(3)

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
7561,7562,141274,10016,28,NaN,0,NaN,F,NaN,NaN,NaN,983,admit,Med-Surg ICU,0.072222,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.0,NaN,0,64.0,NaN,NaN,NaN,NaN,7.0,...,23.0,12.4,12.4,34.9,34.9,1.1,1.1,NaN,NaN,224.0,224.0,4.2,4.2,137.0,137.0,2.6,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
8143,8144,145102,10139,40,23.8,0,Caucasian,F,157.5,NaN,Accident & Emergency,999,admit,Med-Surg ICU,0.050694,0,59.0,2.7,123.0,702.01,0,0,0.3,11.0,0.60,NaN,4.0,6.0,0.0,5.0,285.0,102.0,35.1,0,64.0,NaN,NaN,NaN,NaN,13.0,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.2,3.2,139.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1569,1570,138053,10102,82,28.1,1,Caucasian,M,177.8,Operating Room,Operating Room / Recovery,384,admit,CSICU,5.725000,0,88.9,NaN,308.0,1207.01,1,0,NaN,29.0,1.47,0.95,1.0,1.0,0.0,1.0,211.0,114.0,24.8,1,50.0,35.0,35.0,242.0,7.34,10.0,...,20.0,7.3,7.3,22.1,22.1,2.1,2.1,NaN,NaN,115.0,115.0,4.2,4.2,146.0,146.0,12.9,12.9,45.0,31.0,7.34,7.2,242.0,83.0,314.3,124.0,35.0,35.0,7.34,7.34,242.0,242.0,254.7,254.7,0,0,0,0,0,0,0


In [ ]:
pd.set_option('display.max_rows', None)
test_df.isnull().sum() / test_df.shape[0] * 100

Unnamed: 0                      0.000000
encounter_id                    0.000000
hospital_id                     0.000000
age                             0.000000
bmi                             9.917921
elective_surgery                0.000000
ethnicity                       1.993355
gender                          0.048857
height                          2.941176
hospital_admit_source          26.705101
icu_admit_source                0.244284
icu_id                          0.000000
icu_stay_type                   0.000000
icu_type                        0.000000
pre_icu_los_days                0.000000
readmission_status              0.000000
weight                          8.872386
albumin_apache                 59.165527
apache_2_diagnosis              1.661130
apache_3j_diagnosis             1.055306
apache_post_operative           0.000000
arf_apache                      0.000000
bilirubin_apache               61.657221
bun_apache                     20.138753
creatinine_apach

In [ ]:
test_df.shape

(10234, 180)

In [ ]:
percent_missing_values_attributes = test_df.isnull().sum() / test_df.shape[0] * 100
missing_values_attributes_of_testing = []
for item,value in percent_missing_values_attributes.items():
    if value >= 50:
        missing_values_attributes_of_testing.append(item)
len(missing_values_attributes_of_testing)

74

In [ ]:
set(missing_values_attributes_of_testing) - set(missing_values_attributes)

{'h1_glucose_max', 'h1_glucose_min', 'urineoutput_apache'}

In [ ]:
len(missing_values_attributes)

71

In [ ]:
test_df = test_df.drop(columns = ['Unnamed: 0', 'hospital_id'])

In [ ]:
test_df = test_df.drop(columns = missing_values_attributes)

In [ ]:
set(training_df.columns) - set(test_df.columns)

{'diabetes_mellitus'}

In [ ]:
test_df.sample(5)

,encounter_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,...,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
1382,145361,51,31.3,0,3.0,0.0,154.9,NaN,0.0,273,0,6,0.097917,0,75.0,NaN,217.0,1503.02,1,0,NaN,7.0,0.43,1.0,3.0,6.0,0.0,1.0,142.0,139.0,36.8,1,183.0,42.0,42.0,277.0,7.32,12.0,139.0,35.30,...,NaN,NaN,NaN,NaN,NaN,1.1,1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,42.0,7.37,7.32,277.0,142.0,284.0,277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
4268,141093,86,29.2,0,2.0,0.0,142.2,2.0,0.0,611,0,7,0.102083,0,59.0,3.0,119.0,601.01,0,0,0.9,14.0,0.67,0.4,2.0,5.0,0.0,4.0,111.0,120.0,27.3,1,111.0,29.0,29.0,135.0,7.50,30.0,120.0,36.30,...,NaN,11.4,11.4,32.1,32.1,NaN,NaN,NaN,NaN,217.0,217.0,NaN,NaN,NaN,NaN,21.9,21.9,29.0,29.0,7.50,7.50,135.0,135.0,337.5,337.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
228,140521,58,NaN,1,2.0,1.0,180.3,5.0,2.0,103,0,7,0.381250,0,NaN,NaN,308.0,1207.01,1,0,NaN,NaN,NaN,0.5,4.0,6.0,0.0,5.0,209.0,90.0,32.7,1,47.0,37.0,37.0,72.0,7.37,41.0,NaN,36.22,...,NaN,13.2,13.2,38.5,38.5,1.3,1.2,NaN,NaN,123.0,123.0,3.7,3.7,NaN,NaN,15.7,15.7,40.0,33.0,7.40,7.34,145.0,72.0,248.6,111.4,40.0,37.0,7.38,7.34,145.0,78.0,111.4,111.4,0,0,0,0,0,0,0
6888,140627,78,28.3,0,2.0,0.0,165.1,NaN,0.0,905,0,5,0.155556,0,77.1,3.1,124.0,306.01,0,0,0.6,10.0,0.80,NaN,4.0,6.0,0.0,5.0,98.0,176.0,23.8,0,126.0,NaN,NaN,NaN,NaN,22.0,139.0,36.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
7822,135500,69,31.9,1,2.0,0.0,175.3,NaN,2.0,953,0,5,0.520833,0,97.9,NaN,209.0,1302.02,1,0,NaN,17.0,0.98,NaN,4.0,6.0,0.0,5.0,184.0,95.0,35.5,0,120.0,NaN,NaN,NaN,NaN,31.0,135.0,36.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0


In [ ]:
test_df.shape

(10234, 107)

In [ ]:
for column in test_df.columns:
    if test_df[column].isnull().sum() > 0 and str(test_df[column].dtype).find('64') > -1:
        test_df[column] = fill_missing_value_for_continuous_atrr(test_df, column)

In [ ]:
test_df['ethnicity'] = encode(test_df['ethnicity'])
test_df['gender'] = encode(test_df['gender'])
test_df['hospital_admit_source'] = encode(test_df['hospital_admit_source'])
test_df['icu_admit_source'] = encode(test_df['icu_admit_source'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [ ]:
test_df['ethnicity'] = fill_missing_value_for_categorical_atrr(test_df['ethnicity'])
test_df['gender'] = fill_missing_value_for_categorical_atrr(test_df['gender'])
test_df['hospital_admit_source'] = fill_missing_value_for_categorical_atrr(test_df['hospital_admit_source'])
test_df['icu_admit_source'] = fill_missing_value_for_categorical_atrr(test_df['icu_admit_source'])

In [ ]:
from sklearn import preprocessing
# test_df['icu_stay_type'] = preprocessing.LabelEncoder().fit_transform(test_df['icu_stay_type'])
# test_df['icu_type'] = preprocessing.LabelEncoder().fit_transform(test_df['icu_type'])
test_df['icu_stay_type'] = icu_stay_type_le.transform(test_df['icu_stay_type'])
test_df['icu_type'] = icu_type_le.transform(test_df['icu_type'])

In [ ]:
test_df['ethnicity'] = test_df['ethnicity'].astype('float64')
test_df['gender'] = test_df['gender'].astype("float64")
test_df['hospital_admit_source'] = test_df['hospital_admit_source'].astype("float64")
test_df['icu_admit_source'] = test_df['icu_admit_source'].astype('float64')

In [ ]:
pd.set_option('display.max_rows', None)
test_df.isnull().sum() / test_df.shape[0] * 100

encounter_id                   0.0
age                            0.0
bmi                            0.0
elective_surgery               0.0
ethnicity                      0.0
gender                         0.0
height                         0.0
hospital_admit_source          0.0
icu_admit_source               0.0
icu_id                         0.0
icu_stay_type                  0.0
icu_type                       0.0
pre_icu_los_days               0.0
readmission_status             0.0
weight                         0.0
apache_2_diagnosis             0.0
apache_3j_diagnosis            0.0
apache_post_operative          0.0
arf_apache                     0.0
bun_apache                     0.0
creatinine_apache              0.0
gcs_eyes_apache                0.0
gcs_motor_apache               0.0
gcs_unable_apache              0.0
gcs_verbal_apache              0.0
glucose_apache                 0.0
heart_rate_apache              0.0
hematocrit_apache              0.0
intubated_apache    

In [ ]:
test_df['icu_stay_type'].value_counts()

0    10088
2      121
1       25
Name: icu_stay_type, dtype: int64

In [ ]:
RobustScaler().fit_transform(test_df)

array([[ 0.76880501,  0.29166667,  0.07903366, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.26975774,  0.875     ,  0.07903366, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.27846838,  0.29166667,  0.07903366, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.61292079,  0.375     , -0.74358974, ...,  0.        ,
         0.        ,  0.        ],
       [-0.88041013,  1.04166667, -1.1025641 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.66264404, -1.45833333, -0.37179487, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
import joblib
model_name = 'DT_robust'
filename = 'Models/' + model_name +'.pkl'
loaded_model = joblib.load(filename)
pred_cols = list(test_df.columns.values)[:]
# apply the whole pipeline to data
# result = loaded_model.score(X_test, Y_test)

In [ ]:
RobustScaler().fit_transform(test_df.values)

array([[ 0.76880501,  0.29166667,  0.07903366, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.26975774,  0.875     ,  0.07903366, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.27846838,  0.29166667,  0.07903366, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.61292079,  0.375     , -0.74358974, ...,  0.        ,
         0.        ,  0.        ],
       [-0.88041013,  1.04166667, -1.1025641 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.66264404, -1.45833333, -0.37179487, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
probabilities = xgb_model.predict_proba(robust.transform(test_df.values))

In [ ]:
probabilities = xgb_model.predict_proba(test_df)

In [ ]:
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)
probabilities = loaded_model.predict_proba(test_df)

In [ ]:
classification = loaded_model.predict(test_df)

In [ ]:
result = pd.DataFrame()

In [ ]:
probabilities[:, 1]

array([0.06495849, 0.11591712, 0.18704437, ..., 0.1557009 , 0.04190606,
       0.05991388], dtype=float32)

In [ ]:
result['encounter_id'] = test_df['encounter_id']

In [ ]:
result['diabetes_mellitus'] = probabilities[:, 1]

In [ ]:
result.head(10)

,encounter_id,diabetes_mellitus
0,144740,0.067111
1,141990,0.121921
2,142038,0.166691
3,138628,0.072946
4,141682,0.339903
5,139096,0.126203
6,142994,0.565111
7,141954,0.071628
8,135344,0.597933
9,142552,0.589757


In [ ]:
model_name = 'XGBOOST'

In [ ]:
result.to_csv(model_name+".csv", index = False)